<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=96cdb160eeb9df55178cc8458e449f61b1002d08237e02c932201ac7b018157a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-06 12:17:21
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -1.52 L (-1.05%)
Current PnL: -21.57 L (-14.13%)
CY Booked + Current PnL: -8.92 L (-5.85%)
-------------------
Total profit:  1.91 L
Total loss:  -23.48 L
-------------------
Total Booked + Current PnL: 18.06 L (14.38%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.85%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.55 L (61.93%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,67.0,X-LC,5.81,233236.0,22870.0,5901.0,1.49,10.87,2.53,13.68,37.0,3.88,1.64,29.19,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,101.92,52.0,M-SC,1.75,86984.0,-13793.0,13883.0,-0.72,-13.69,15.96,0.09,245.0,-0.99,0.61,14.28,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,64.0,M-LC,10.63,170721.0,29881.0,14614.0,-0.92,21.22,8.56,31.59,66.0,2.04,1.20,53.32,XY24,NTT,BANKS
2,ABBOTINDIA,35195.00,-9.64,47.0,X-MC,2.56,89280.0,-1398.0,16303.0,1.36,-1.54,18.26,16.44,101.0,-0.09,0.63,18.32,X40,ATH,PHARMA
33,ICICIGI,2252.93,-13.61,57.0,X-MC,5.55,148474.0,12341.0,18247.0,-1.65,9.07,12.29,22.47,91.0,0.68,1.05,23.22,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-15.63,58.0,M-SC,27.26,81187.0,-95628.0,176427.0,4.92,-54.08,217.31,45.70,197.0,-0.54,0.57,5.99,XY24,NTT,TRAVEL
6,ASIANPAINT,3465.66,-9.14,69.0,X-LC,4.63,229416.0,-22352.0,75570.0,4.84,-8.88,32.94,21.13,27.0,-0.30,1.62,23.40,X40,ATH,PAINTS
67,SIS,528.00,2025.39,46.0,H-SC,2.73,85782.0,-25250.0,48861.0,1.63,-22.74,56.96,21.26,156.0,-0.52,0.60,15.90,OX40N,NTT,MISC
58,RELIANCE,1533.00,-6.91,67.0,X-LC,5.81,233236.0,22870.0,5901.0,1.49,10.87,2.53,13.68,37.0,3.88,1.64,29.19,XY25,NTT,REFINERIES
13,BERGEPAINT,680.00,-17.83,55.0,X-MC,1.27,220786.0,-6553.0,54622.0,1.43,-2.88,24.74,21.14,106.0,-0.12,1.56,24.45,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,32.84,48.0,M-SC,4.33,175492.0,-86726.0,156556.0,-5.17,-33.07,89.21,26.63,229.0,-0.55,1.24,9.11,XY24,NTT,MISC
8,ATULAUTO,844.00,3364.86,35.0,M-SC,3.28,155635.0,-41497.0,131325.0,-4.21,-21.05,84.38,45.57,236.0,-0.32,1.10,10.34,XY24,NTT,AUTO
84,VOLTAS,1530.00,-6.39,31.0,X-MC,4.49,196350.0,4608.0,33144.0,-3.71,2.40,16.88,19.69,99.0,0.14,1.38,9.36,XY25,NTT,AC
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.38,129251.0,3954.0,54970.0,-3.60,3.16,42.53,47.03,141.0,0.07,0.91,61.83,MH,ATH,POWER
76,TITAGARH,1548.00,-9.33,40.0,H-SC,3.36,193720.0,-48457.0,159218.0,-3.37,-20.01,82.19,45.74,158.0,-0.30,1.36,26.00,XY24,NTT,ENGINEERING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.92,52.0,M-SC,1.75,86984.0,-13793.0,13883.0,-0.72,-13.69,15.96,0.09,245.0,-0.99,0.61,14.28,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.67,221418.0,-48249.0,84582.0,-1.01,-17.89,38.20,13.47,138.0,-0.57,1.56,12.58,XY24,NTT,PAINTS
17,CERA,9475.0,-21.96,53.0,H-SC,1.76,142278.0,-33625.0,75649.0,-1.72,-19.12,53.17,23.89,149.0,-0.44,1.00,22.88,OX40N,NTT,CERAMICS
67,SIS,528.0,2025.39,46.0,H-SC,2.73,85782.0,-25250.0,48861.0,1.63,-22.74,56.96,21.26,156.0,-0.52,0.60,15.90,OX40N,NTT,MISC
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.52,97877.0,-29928.0,29911.0,0.02,-23.42,30.56,-0.01,232.0,-1.00,0.69,10.83,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,64.0,M-LC,10.63,170721.0,29881.0,14614.0,-0.92,21.22,8.56,31.59,66.0,2.04,1.2,53.32,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.16,57.0,M-MC,9.92,235570.0,10608.0,157055.0,-1.00,4.72,66.67,74.53,192.0,0.07,1.66,38.49,XY24,BTT,STEEL
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.38,129251.0,3954.0,54970.0,-3.60,3.16,42.53,47.03,141.0,0.07,0.91,61.83,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,64.0,M-LC,10.63,170721.0,29881.0,14614.0,-0.92,21.22,8.56,31.59,66.0,2.04,1.20,53.32,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.36,124274.0,9476.0,84568.0,-1.72,8.25,68.05,81.92,223.0,0.11,0.88,47.12,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,54.0,H-SC,5.85,130189.0,6853.0,124773.0,-0.48,5.56,95.84,106.72,119.0,0.05,0.92,30.23,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.36,124274.0,9476.0,84568.0,-1.72,8.25,68.05,81.92,223.0,0.11,0.88,47.12,XR,NTT,DURABLES
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.38,129251.0,3954.0,54970.0,-3.60,3.16,42.53,47.03,141.0,0.07,0.91,61.83,MH,ATH,POWER
36,INDIAMART,4810.62,-53.33,54.0,H-SC,5.85,130189.0,6853.0,124773.0,-0.48,5.56,95.84,106.72,119.0,0.05,0.92,30.23,AR,ATH,MISC
86,WIPRO,420.00,-15.61,41.0,M-LC,5.50,148186.0,-2759.0,112206.0,0.46,-1.83,75.72,72.51,53.0,-0.02,1.04,4.08,XR,NTT,IT
31,HINDZINC,730.22,24.68,43.0,M-LC,10.42,202993.0,-2083.0,114650.0,-1.42,-1.02,56.48,54.89,52.0,-0.02,1.43,22.83,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.04,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.94,87416.0,-41254.0,86551.0,-1.21,-32.06,99.01,35.20,219.0,-0.48,0.62,0.33,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-23.55,28.0,X-MC,4.69,191768.0,-8579.0,39082.0,-1.93,-4.28,20.38,15.23,178.0,-0.22,1.35,26.59,X40N,NTT,REALTY
84,VOLTAS,1530.00,-6.39,31.0,X-MC,4.49,196350.0,4608.0,33144.0,-3.71,2.40,16.88,19.69,99.0,0.14,1.38,9.36,XY25,NTT,AC
23,DMART,5391.45,-19.03,32.0,X-LC,8.30,206385.0,-4022.0,63195.0,-1.31,-1.91,30.62,28.12,38.0,-0.06,1.45,21.26,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.21,36.0,X-MC,0.91,199305.0,-1515.0,69318.0,-0.85,-0.75,34.78,33.76,80.0,-0.02,1.40,4.40,X40,ATH,FMCG
41,ITC,452.00,-38.34,45.0,X-LC,1.13,198974.0,-1164.0,22046.0,-0.49,-0.58,11.08,10.44,4.0,-0.05,1.40,4.95,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.90,43.0,H-LC,1.25,154550.0,-31545.0,79021.0,-0.92,-16.95,51.13,25.51,15.0,-0.40,1.09,13.89,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.30,37.0,X-MC,3.06,305970.0,-22021.0,128569.0,-1.98,-6.71,42.02,32.48,92.0,-0.17,2.16,3.94,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.52,65.0,X-LC,4.12,245134.0,10990.0,37628.0,-1.77,4.69,15.35,20.77,86.0,0.29,1.73,11.94,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.76,48986.0,-16020.0,166949.0,-2.69,-24.64,340.81,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.94,87416.0,-41254.0,86551.0,-1.21,-32.06,99.01,35.20,219.0,-0.48,0.62,0.33,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.48,33.0,X-MC,7.24,79690.0,-3820.0,23851.0,-0.72,-4.57,29.93,23.98,82.0,-0.16,0.56,0.98,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.74,41.0,X-MC,7.34,217530.0,-45835.0,155164.0,-0.71,-17.40,71.33,41.51,84.0,-0.30,1.53,2.08,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,44.0,X-MC,2.44,183300.0,-54651.0,207294.0,0.02,-22.97,113.09,64.15,174.0,-0.26,1.29,2.82,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.76,48986.0,-16020.0,166949.0,-2.69,-24.64,340.81,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.48,33.0,X-MC,7.24,79690.0,-3820.0,23851.0,-0.72,-4.57,29.93,23.98,82.0,-0.16,0.56,0.98,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.46,41.0,X-SC,4.31,81207.0,-63953.0,139879.0,-0.29,-44.06,172.25,52.31,136.0,-0.46,0.57,8.70,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.94,87416.0,-41254.0,86551.0,-1.21,-32.06,99.01,35.20,219.0,-0.48,0.62,0.33,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.64,47.0,X-MC,2.56,89280.0,-1398.0,16303.0,1.36,-1.54,18.26,16.44,101.0,-0.09,0.63,18.32,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.01,48.0,X-LC,9.86,282555.0,-63401.0,130117.0,0.53,-18.33,46.05,19.28,1.0,-0.49,1.99,4.46,X40,ATH,IT
39,INFY,2275.00,-19.60,49.0,X-LC,5.12,313408.0,520.0,171152.0,0.24,0.17,54.61,54.87,3.0,0.00,2.21,8.46,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,1.13,198974.0,-1164.0,22046.0,-0.49,-0.58,11.08,10.44,4.0,-0.05,1.40,4.95,X40,NTT,FMCG
83,VBL,671.64,-16.58,53.0,X-LC,6.88,300402.0,-15440.0,128091.0,-0.03,-4.89,42.64,35.67,5.0,-0.12,2.12,8.29,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,51.0,H-LC,5.09,250944.0,-10675.0,129889.0,-0.31,-4.08,51.76,45.57,7.0,-0.08,1.77,6.78,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,36.0,L-SC,27.76,75456.0,-38093.0,78142.0,-2.89,-33.55,103.56,35.27,268.0,-0.49,0.53,84.68,XR,NTT,HOTELS
7,ASIANTILES,137.00,7438.89,67.0,L-SC,10.63,82689.0,-11121.0,87741.0,-1.01,-11.85,106.11,81.67,269.0,-0.13,0.58,60.13,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.60,49.0,H-SC,8.36,91605.0,-6375.0,27976.0,-2.38,-6.51,30.54,22.05,152.0,-0.23,0.65,33.15,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.36,124274.0,9476.0,84568.0,-1.72,8.25,68.05,81.92,223.0,0.11,0.88,47.12,XR,NTT,DURABLES
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.38,129251.0,3954.0,54970.0,-3.60,3.16,42.53,47.03,141.0,0.07,0.91,61.83,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-53.06,67.0,H-SC,7.29,269819.0,-16686.0,274028.0,-3.02,-5.82,101.56,89.82,134.0,-0.06,1.90,42.75,XY24,NTT,FINANCE
81,VAIBHAVGBL,521.00,74.10,64.0,H-SC,2.30,150814.0,-31961.0,143032.0,-2.60,-17.49,94.84,60.77,125.0,-0.22,1.06,35.08,XR,NTT,JEWELLERY
62,SAMMAANCAP,326.00,-197.73,71.0,M-SC,33.11,168318.0,19098.0,125077.0,0.43,12.80,74.31,96.62,208.0,0.15,1.19,82.16,XY25,NTT,FINANCE
68,SONACOMS,806.63,-32.06,66.0,M-SC,6.91,84499.0,-16660.0,56665.0,-0.20,-16.47,67.06,39.54,202.0,-0.29,0.60,19.15,AR,ATH,AUTO
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.38,129251.0,3954.0,54970.0,-3.60,3.16,42.53,47.03,141.0,0.07,0.91,61.83,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.74
1,25,44.43
2,50,75.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.76
LC    33.24
MC    23.02
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.93
X40      16.29
X40N     12.43
XY25     11.96
XR        9.83
AR        7.86
OX40N     6.54
X200      1.79
X5K       1.43
SR        1.05
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.19
X-LC    22.53
X-MC    18.73
M-SC    12.64
X-SC     5.49
M-LC     5.08
H-LC     4.65
H-MC     2.32
M-MC     1.66
L-SC     1.44
L-LC     0.98
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,-3.93,37.53
IT,12.44,-19.00,82.75
FINANCE,12.19,-9.38,58.96
MISC,7.72,-18.81,78.73
ELECTRICAL,5.78,-10.22,49.29
PAINTS,5.61,-16.22,33.55
BANKS,4.00,-18.65,87.67
INSURANCE,3.84,-1.53,39.38
AUTO,2.74,-48.26,110.14


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3325730.0,22
XR,1260452.0,13
AR,1173833.0,9
X40,845869.0,12
X40N,703986.0,9
OX40N,556179.0,9
XY25,476576.0,8
SR,272835.0,2
X5K,114650.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3397185.0,24
M-SC,1255294.0,15
X-MC,1214938.0,14
X-LC,1082131.0,13
X-SC,570445.0,6
M-LC,286426.0,4
H-LC,279251.0,3
H-MC,257521.0,2
L-SC,256036.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1194213.0      6
           AR         840802.0      5
           XR         790446.0      7
M-SC       XY24       724592.0      6
X-MC       XY24       569975.0      4
X-LC       X40        460790.0      5
X-SC       X40N       316945.0      4
X-LC       XY24       315185.0      2
X-MC       X40        298528.0      6
H-SC       SR         272835.0      2
           OX40N      243919.0      3
X-LC       X40N       228914.0      3
H-LC       AR         208910.0      2
X-MC       XY25       188308.0      2
H-MC       XY24       183147.0      1
X-SC       XY24       166949.0      1
L-SC       XR         165883.0      2
X-MC       X40N       158127.0      2
M-MC       XY24       157055.0      1
M-SC       OX40N      147733.0      4
           XR         133771.0      2
           XY25       125077.0      1
           AR         124121.0      2
M-LC       X5K        114650.0      1
           XR         112206.0      1
L-SC       OX40N       90153.0      1
X-SC       X40         86551.0      1
X-LC       XY25        77242.0      3
H-MC       OX40N       74374.0      1
H-LC       X200        70341.0      1
L-MC       XR          58146.0      1
H-SC       MH          54970.0      1
M-LC       XY25        44956.0      1
L-LC       XY25        40993.0      1
M-LC       XY24        14614.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
